In [2]:
import pickle
import numpy as np
import cv2
import mediapipe as mp
import warnings

In [ ]:
warnings.filterwarnings("ignore", category=UserWarning, module='google.protobuf')

# Load model
model_dict = pickle.load(open('../model/rf_model_8.p', 'rb'))
model = model_dict['model']

# Definisikan labels_dict untuk mapping label numerik ke huruf
class_labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
labels_dict = {i: class_labels[i] for i in range(len(class_labels))}
print("Labels Dictionary:", labels_dict)  # Menampilkan dictionary untuk memastikan benar

cap = cv2.VideoCapture(0)

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

while True:
    data_aux = []
    x_ = []
    y_ = []

    ret, frame = cap.read()

    H, W, _ = frame.shape

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = hands.process(frame_rgb)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())

            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y

                x_.append(x)
                y_.append(y)

            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y
                data_aux.append(x - min(x_))
                data_aux.append(y - min(y_))

        x1 = int(min(x_) * W) - 10
        y1 = int(min(y_) * H) - 10

        x2 = int(max(x_) * W) - 10
        y2 = int(max(y_) * H) - 10

        prediction = model.predict([np.asarray(data_aux)])

        # Cek apakah hasil prediksi merupakan string
        if isinstance(prediction[0], str):
            predicted_character = prediction[0]  # Jika sudah berupa string
        else:
            predicted_character = labels_dict[int(prediction[0])]  # Jika berupa indeks numerik

        # Define padding and position for the label
        padding = 5
        text_size, _ = cv2.getTextSize(predicted_character, cv2.FONT_HERSHEY_PLAIN, 1.3, 1)
        label_x1 = x1
        label_y1 = y1 - text_size[1] - padding * 2
        label_x2 = x1 + text_size[0] + padding * 2
        label_y2 = y1

        # Draw a filled rectangle for the label background
        cv2.rectangle(frame, (label_x1, label_y1), (label_x2, label_y2), (0, 0, 0), cv2.FILLED)

        # Draw the rectangle around the detected hand
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 4)

        # Put the text on the label
        cv2.putText(frame, predicted_character, (label_x1 + padding, label_y2 - padding),
                    cv2.FONT_HERSHEY_PLAIN, 1.3, (255, 255, 255), 1, cv2.LINE_AA)

    cv2.imshow('frame', frame)
    cv2.waitKey(1)

cap.release()
cv2.destroyAllWindows()

Labels Dictionary: {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J', 10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'O', 15: 'P', 16: 'Q', 17: 'R', 18: 'S', 19: 'T', 20: 'U', 21: 'V', 22: 'W', 23: 'X', 24: 'Y', 25: 'Z'}
